In the Prevoius notebook , we have seen how to write file as delta format.Now will see how data formated as delta table

In [0]:
%sql

create schema if not exists delta;

In [0]:
%sql

create Table `delta`.deltaFile (
    Education_Level VARCHAR(50),
    Line_Number INT,
    Employed INT,
    Unemployed INT,
    Industry VARCHAR(50),
    Gender VARCHAR(10),
    Date_Inserted DATE,
    dense_rank INT
)

Any Table you are creating by default is a DELTA TABLE IN DATABRICKS and Treated as Managed Table

Managed Table : Means that Meta-Data and Data of Table maintained by the DataBricks 

In [0]:
dbutils.fs.ls('dbfs:/user/hive/warehouse/delta.db/deltafile') #The presence of delta_log says that it's a delta table 

[FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log/', name='_delta_log/', size=0, modificationTime=1709018399000)]

In [0]:
#lets see what delta_logs contains : 

dbutils.fs.ls('dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log')

[FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log/00000000000000000000.crc', name='00000000000000000000.crc', size=2606, modificationTime=1709018401000),
 FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log/00000000000000000000.json', name='00000000000000000000.json', size=1624, modificationTime=1709018400000),
 FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log/__tmp_path_dir/', name='__tmp_path_dir/', size=0, modificationTime=1709018399000)]

In [0]:
%sql

select * from json.`dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log/00000000000000000000.json`
-- or 
-- select * from text.`dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log/00000000000000000000.json`

commitInfo,metaData,protocol
"List(1227-053858-m1tsqg19, Databricks-Runtime/13.3.x-photon-scala2.12, true, WriteSerializable, List(482608276354729), CREATE TABLE, List(null, true, [], {}, false), List(false), 1709018399912, 8a3d7bd9-7644-4b01-a61f-dbd5a76dcddf, 4700893722167418, lokeswar.valluru@xyenta.com)",null,null
null,"List(1709018399839, List(parquet), 995e51d4-9ff8-4bbd-a227-cbbc4dbc65e2, List(), {""type"":""struct"",""fields"":[{""name"":""Education_Level"",""type"":""string"",""nullable"":true,""metadata"":{""__CHAR_VARCHAR_TYPE_STRING"":""varchar(50)""}},{""name"":""Line_Number"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Employed"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Unemployed"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Industry"",""type"":""string"",""nullable"":true,""metadata"":{""__CHAR_VARCHAR_TYPE_STRING"":""varchar(50)""}},{""name"":""Gender"",""type"":""string"",""nullable"":true,""metadata"":{""__CHAR_VARCHAR_TYPE_STRING"":""varchar(10)""}},{""name"":""Date_Inserted"",""type"":""date"",""nullable"":true,""metadata"":{}},{""name"":""dense_rank"",""type"":""integer"",""nullable"":true,""metadata"":{}}]})",null
null,null,"List(1, 2)"


Lets Add records to delta Table . Main varaibles u need to look in above Query is Operation 

In [0]:
%sql
INSERT INTO `delta`.deltafile
VALUES
    ('Bachelor', 1, 4500, 500, 'IT', 'Male', '2023-07-12',  1),
    ('Master', 2, 6500, 500, 'Finance', 'Female', '2023-07-12', 2),
    ('High School', 3, 3500, 500, 'Retail', 'Male', '2023-07-12', 3),
    ('PhD', 4, 5500, 500, 'Healthcare', 'Female', '2023-07-12', 4);


num_affected_rows,num_inserted_rows
4,4


Now Querying the Table delta.deltafile

In [0]:
%sql

select * from `delta`.deltafile

Education_Level,Line_Number,Employed,Unemployed,Industry,Gender,Date_Inserted,dense_rank
Bachelor,1,4500,500,IT,Male,2023-07-12,1
Master,2,6500,500,Finance,Female,2023-07-12,2
High School,3,3500,500,Retail,Male,2023-07-12,3
PhD,4,5500,500,Healthcare,Female,2023-07-12,4


In [0]:
dbutils.fs.ls('dbfs:/user/hive/warehouse/delta.db/deltafile') #now we inserted data so we got parquet files. 

[FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log/', name='_delta_log/', size=0, modificationTime=1709018399000),
 FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/part-00000-34241875-df11-4d53-9ee6-c93fac83081b.c000.snappy.parquet', name='part-00000-34241875-df11-4d53-9ee6-c93fac83081b.c000.snappy.parquet', size=2239, modificationTime=1709019075000)]

In [0]:
dbutils.fs.ls('dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log')

[FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log/00000000000000000000.crc', name='00000000000000000000.crc', size=2606, modificationTime=1709018401000),
 FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log/00000000000000000000.json', name='00000000000000000000.json', size=1624, modificationTime=1709018400000),
 FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log/00000000000000000001.crc', name='00000000000000000001.crc', size=3511, modificationTime=1709019076000),
 FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log/00000000000000000001.json', name='00000000000000000001.json', size=1495, modificationTime=1709019075000),
 FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log/__tmp_path_dir/', name='__tmp_path_dir/', size=0, modificationTime=1709018399000)]


## Number of Json / parquet files depend on Number of operations  

In [0]:
%sql

select * from text .`dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log/00000000000000000001.json`

value
"{""commitInfo"":{""timestamp"":1709019075447,""userId"":""4700893722167418"",""userName"":""lokeswar.valluru@xyenta.com"",""operation"":""WRITE"",""operationParameters"":{""mode"":""Append"",""statsOnLoad"":false,""partitionBy"":""[]""},""notebook"":{""notebookId"":""482608276354729""},""clusterId"":""1227-053858-m1tsqg19"",""readVersion"":0,""isolationLevel"":""WriteSerializable"",""isBlindAppend"":true,""operationMetrics"":{""numFiles"":""1"",""numOutputRows"":""4"",""numOutputBytes"":""2239""},""tags"":{""restoresDeletedRows"":""false""},""engineInfo"":""Databricks-Runtime/13.3.x-photon-scala2.12"",""txnId"":""1adc6c5e-c5a2-4db4-ace0-aa0b1385c485""}}"
"{""add"":{""path"":""part-00000-34241875-df11-4d53-9ee6-c93fac83081b.c000.snappy.parquet"",""partitionValues"":{},""size"":2239,""modificationTime"":1709019075000,""dataChange"":true,""stats"":""{\""numRecords\"":4,\""minValues\"":{\""Education_Level\"":\""Bachelor\"",\""Line_Number\"":1,\""Employed\"":3500,\""Unemployed\"":500,\""Industry\"":\""Finance\"",\""Gender\"":\""Female\"",\""Date_Inserted\"":\""2023-07-12\"",\""dense_rank\"":1},\""maxValues\"":{\""Education_Level\"":\""PhD\"",\""Line_Number\"":4,\""Employed\"":6500,\""Unemployed\"":500,\""Industry\"":\""Retail\"",\""Gender\"":\""Male\"",\""Date_Inserted\"":\""2023-07-12\"",\""dense_rank\"":4},\""nullCount\"":{\""Education_Level\"":0,\""Line_Number\"":0,\""Employed\"":0,\""Unemployed\"":0,\""Industry\"":0,\""Gender\"":0,\""Date_Inserted\"":0,\""dense_rank\"":0}}"",""tags"":{""INSERTION_TIME"":""1709019075000000"",""MIN_INSERTION_TIME"":""1709019075000000"",""MAX_INSERTION_TIME"":""1709019075000000"",""OPTIMIZE_TARGET_SIZE"":""268435456""}}}"


Updating the Table 

In [0]:
%sql 
update `delta`.deltafile
set Industry = 'Finance' where Education_Level = 'PhD'

num_affected_rows
1


In [0]:
%sql 

select * from `delta`.deltafile

Education_Level,Line_Number,Employed,Unemployed,Industry,Gender,Date_Inserted,dense_rank
Bachelor,1,4500,500,IT,Male,2023-07-12,1
Master,2,6500,500,Finance,Female,2023-07-12,2
High School,3,3500,500,Retail,Male,2023-07-12,3
PhD,4,5500,500,Finance,Female,2023-07-12,4


In [0]:
dbutils.fs.ls('dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log')

[FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log/00000000000000000000.crc', name='00000000000000000000.crc', size=2606, modificationTime=1709018401000),
 FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log/00000000000000000000.json', name='00000000000000000000.json', size=1624, modificationTime=1709018400000),
 FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log/00000000000000000001.crc', name='00000000000000000001.crc', size=3511, modificationTime=1709019076000),
 FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log/00000000000000000001.json', name='00000000000000000001.json', size=1495, modificationTime=1709019075000),
 FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log/00000000000000000002.crc', name='00000000000000000002.crc', size=3511, modificationTime=1709019388000),
 FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log/00000000000000000002.json', name='0000000000

In [0]:
%sql 
select * from Text.`dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log/00000000000000000002.json`

value
"{""commitInfo"":{""timestamp"":1709019386915,""userId"":""4700893722167418"",""userName"":""lokeswar.valluru@xyenta.com"",""operation"":""UPDATE"",""operationParameters"":{""predicate"":""[\""(Education_Level#4543 = PhD)\""]""},""notebook"":{""notebookId"":""482608276354729""},""clusterId"":""1227-053858-m1tsqg19"",""readVersion"":1,""isolationLevel"":""WriteSerializable"",""isBlindAppend"":false,""operationMetrics"":{""numRemovedFiles"":""1"",""numRemovedBytes"":""2239"",""numCopiedRows"":""3"",""numDeletionVectorsAdded"":""0"",""numDeletionVectorsRemoved"":""0"",""numAddedChangeFiles"":""0"",""executionTimeMs"":""1514"",""scanTimeMs"":""843"",""numAddedFiles"":""1"",""numUpdatedRows"":""1"",""numAddedBytes"":""2296"",""rewriteTimeMs"":""651""},""tags"":{""noRowsCopied"":""false"",""delta.rowTracking.preserved"":""false"",""restoresDeletedRows"":""false""},""engineInfo"":""Databricks-Runtime/13.3.x-photon-scala2.12"",""txnId"":""01b35a97-7ed3-4c2d-80b0-cfd5ead7032a""}}"
"{""remove"":{""path"":""part-00000-34241875-df11-4d53-9ee6-c93fac83081b.c000.snappy.parquet"",""deletionTimestamp"":1709019386898,""dataChange"":true,""extendedFileMetadata"":true,""partitionValues"":{},""size"":2239,""tags"":{""INSERTION_TIME"":""1709019075000000"",""MIN_INSERTION_TIME"":""1709019075000000"",""MAX_INSERTION_TIME"":""1709019075000000"",""OPTIMIZE_TARGET_SIZE"":""268435456""}}}"
"{""add"":{""path"":""part-00000-29fc0fc4-a7f1-47e7-9e40-84642ecf1e7b.c000.snappy.parquet"",""partitionValues"":{},""size"":2296,""modificationTime"":1709019386000,""dataChange"":true,""stats"":""{\""numRecords\"":4,\""minValues\"":{\""Education_Level\"":\""Bachelor\"",\""Line_Number\"":1,\""Employed\"":3500,\""Unemployed\"":500,\""Industry\"":\""Finance\"",\""Gender\"":\""Female\"",\""Date_Inserted\"":\""2023-07-12\"",\""dense_rank\"":1},\""maxValues\"":{\""Education_Level\"":\""PhD\"",\""Line_Number\"":4,\""Employed\"":6500,\""Unemployed\"":500,\""Industry\"":\""Retail\"",\""Gender\"":\""Male\"",\""Date_Inserted\"":\""2023-07-12\"",\""dense_rank\"":4},\""nullCount\"":{\""Education_Level\"":0,\""Line_Number\"":0,\""Employed\"":0,\""Unemployed\"":0,\""Industry\"":0,\""Gender\"":0,\""Date_Inserted\"":0,\""dense_rank\"":0}}"",""tags"":{""MAX_INSERTION_TIME"":""1709019075000000"",""INSERTION_TIME"":""1709019075000000"",""MIN_INSERTION_TIME"":""1709019075000000"",""OPTIMIZE_TARGET_SIZE"":""268435456""}}}"


In [0]:
dbutils.fs.ls('dbfs:/user/hive/warehouse/delta.db/deltafile/')

[FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log/', name='_delta_log/', size=0, modificationTime=1709018399000),
 FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/part-00000-29fc0fc4-a7f1-47e7-9e40-84642ecf1e7b.c000.snappy.parquet', name='part-00000-29fc0fc4-a7f1-47e7-9e40-84642ecf1e7b.c000.snappy.parquet', size=2296, modificationTime=1709019386000),
 FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/part-00000-34241875-df11-4d53-9ee6-c93fac83081b.c000.snappy.parquet', name='part-00000-34241875-df11-4d53-9ee6-c93fac83081b.c000.snappy.parquet', size=2239, modificationTime=1709019075000)]

delete operation 



In [0]:
%sql

Delete from `delta`.deltafile
where dense_rank = 2

num_affected_rows
1


In [0]:
dbutils.fs.ls('dbfs:/user/hive/warehouse/delta.db/deltafile/') #a new parquet and json will created after an operation 

[FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log/', name='_delta_log/', size=0, modificationTime=1709018399000),
 FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/part-00000-29fc0fc4-a7f1-47e7-9e40-84642ecf1e7b.c000.snappy.parquet', name='part-00000-29fc0fc4-a7f1-47e7-9e40-84642ecf1e7b.c000.snappy.parquet', size=2296, modificationTime=1709019386000),
 FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/part-00000-34241875-df11-4d53-9ee6-c93fac83081b.c000.snappy.parquet', name='part-00000-34241875-df11-4d53-9ee6-c93fac83081b.c000.snappy.parquet', size=2239, modificationTime=1709019075000),
 FileInfo(path='dbfs:/user/hive/warehouse/delta.db/deltafile/part-00000-b7cc85d0-db11-4f2a-883f-42d57ed165b6.c000.snappy.parquet', name='part-00000-b7cc85d0-db11-4f2a-883f-42d57ed165b6.c000.snappy.parquet', size=2244, modificationTime=1709019757000)]

In [0]:
%sql 
select * from Text.`dbfs:/user/hive/warehouse/delta.db/deltafile/_delta_log/00000000000000000003.json`

value
"{""commitInfo"":{""timestamp"":1709019757681,""userId"":""4700893722167418"",""userName"":""lokeswar.valluru@xyenta.com"",""operation"":""DELETE"",""operationParameters"":{""predicate"":""[\""(dense_rank#5373 = 2)\""]""},""notebook"":{""notebookId"":""482608276354729""},""clusterId"":""1227-053858-m1tsqg19"",""readVersion"":2,""isolationLevel"":""WriteSerializable"",""isBlindAppend"":false,""operationMetrics"":{""numRemovedFiles"":""1"",""numRemovedBytes"":""2296"",""numCopiedRows"":""3"",""numDeletionVectorsAdded"":""0"",""numDeletionVectorsRemoved"":""0"",""numAddedChangeFiles"":""0"",""executionTimeMs"":""979"",""numDeletedRows"":""1"",""scanTimeMs"":""422"",""numAddedFiles"":""1"",""numAddedBytes"":""2244"",""rewriteTimeMs"":""557""},""tags"":{""noRowsCopied"":""false"",""delta.rowTracking.preserved"":""false"",""restoresDeletedRows"":""false""},""engineInfo"":""Databricks-Runtime/13.3.x-photon-scala2.12"",""txnId"":""12d0a249-53ef-4530-a155-b1dcf291dfd6""}}"
"{""remove"":{""path"":""part-00000-29fc0fc4-a7f1-47e7-9e40-84642ecf1e7b.c000.snappy.parquet"",""deletionTimestamp"":1709019757679,""dataChange"":true,""extendedFileMetadata"":true,""partitionValues"":{},""size"":2296,""tags"":{""MAX_INSERTION_TIME"":""1709019075000000"",""INSERTION_TIME"":""1709019075000000"",""MIN_INSERTION_TIME"":""1709019075000000"",""OPTIMIZE_TARGET_SIZE"":""268435456""}}}"
"{""add"":{""path"":""part-00000-b7cc85d0-db11-4f2a-883f-42d57ed165b6.c000.snappy.parquet"",""partitionValues"":{},""size"":2244,""modificationTime"":1709019757000,""dataChange"":true,""stats"":""{\""numRecords\"":3,\""minValues\"":{\""Education_Level\"":\""Bachelor\"",\""Line_Number\"":1,\""Employed\"":3500,\""Unemployed\"":500,\""Industry\"":\""Finance\"",\""Gender\"":\""Female\"",\""Date_Inserted\"":\""2023-07-12\"",\""dense_rank\"":1},\""maxValues\"":{\""Education_Level\"":\""PhD\"",\""Line_Number\"":4,\""Employed\"":5500,\""Unemployed\"":500,\""Industry\"":\""Retail\"",\""Gender\"":\""Male\"",\""Date_Inserted\"":\""2023-07-12\"",\""dense_rank\"":4},\""nullCount\"":{\""Education_Level\"":0,\""Line_Number\"":0,\""Employed\"":0,\""Unemployed\"":0,\""Industry\"":0,\""Gender\"":0,\""Date_Inserted\"":0,\""dense_rank\"":0}}"",""tags"":{""MAX_INSERTION_TIME"":""1709019075000000"",""INSERTION_TIME"":""1709019075000000"",""MIN_INSERTION_TIME"":""1709019075000000"",""OPTIMIZE_TARGET_SIZE"":""268435456""}}}"


In [0]:
%sql

select * from `delta`.deltafile

Education_Level,Line_Number,Employed,Unemployed,Industry,Gender,Date_Inserted,dense_rank
Bachelor,1,4500,500,IT,Male,2023-07-12,1
High School,3,3500,500,Retail,Male,2023-07-12,3
PhD,4,5500,500,Finance,Female,2023-07-12,4
